In [9]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [17]:

cap = cv2.VideoCapture('../raw_data/jiggle_short.mp4')
## Setup mediapipe instance

pl_list = []
with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        
        ret, frame = cap.read()
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            for id, lm in enumerate(landmarks):
                pl_list.append([id, lm.x, lm.y])
            print(landmarks)
        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

[x: 0.48307690024375916
y: 0.4967958331108093
z: -0.14346687495708466
visibility: 0.9917281866073608
, x: 0.48472118377685547
y: 0.498517781496048
z: -0.15904463827610016
visibility: 0.9867081642150879
, x: 0.48652735352516174
y: 0.4990350008010864
z: -0.15890614688396454
visibility: 0.9877869486808777
, x: 0.4883361756801605
y: 0.49949467182159424
z: -0.1588432788848877
visibility: 0.9884873628616333
, x: 0.4807170331478119
y: 0.49929922819137573
z: -0.16360530257225037
visibility: 0.9878612160682678
, x: 0.479480117559433
y: 0.5003727674484253
z: -0.16353678703308105
visibility: 0.9879406690597534
, x: 0.4782119691371918
y: 0.5014791488647461
z: -0.1634540855884552
visibility: 0.9875674247741699
, x: 0.4953361451625824
y: 0.5024909973144531
z: -0.19177338480949402
visibility: 0.976531982421875
, x: 0.4921305179595947
y: 0.5025776624679565
z: -0.21233835816383362
visibility: 0.9593752026557922
, x: 0.48891931772232056
y: 0.4975716769695282
z: -0.14928480982780457
visibility: 0.9871375

[x: 0.5221567749977112
y: 0.3598063886165619
z: -0.05527046322822571
visibility: 0.9955989718437195
, x: 0.5218245983123779
y: 0.35119009017944336
z: -0.044169045984745026
visibility: 0.9929304122924805
, x: 0.5219686627388
y: 0.3513190746307373
z: -0.044184159487485886
visibility: 0.9935026168823242
, x: 0.5220441222190857
y: 0.3514595627784729
z: -0.04420304298400879
visibility: 0.9938752055168152
, x: 0.5195831060409546
y: 0.3503330647945404
z: -0.0553184375166893
visibility: 0.9935404062271118
, x: 0.5178775191307068
y: 0.3500816524028778
z: -0.055323511362075806
visibility: 0.99358069896698
, x: 0.5160311460494995
y: 0.3496221899986267
z: -0.05535765737295151
visibility: 0.9933803677558899
, x: 0.5176288485527039
y: 0.3545331656932831
z: 0.015502266585826874
visibility: 0.9875227808952332
, x: 0.5092529654502869
y: 0.35071346163749695
z: -0.03568030893802643
visibility: 0.9784014225006104
, x: 0.5204921364784241
y: 0.3683583438396454
z: -0.03229328244924545
visibility: 0.993161737

[x: 0.5330546498298645
y: 0.37923702597618103
z: -0.05492096021771431
visibility: 0.9972656965255737
, x: 0.5315335988998413
y: 0.3706219494342804
z: -0.047021299600601196
visibility: 0.9955113530158997
, x: 0.5312314033508301
y: 0.3704507648944855
z: -0.04703439772129059
visibility: 0.9958393573760986
, x: 0.5308545231819153
y: 0.3702370822429657
z: -0.04703512042760849
visibility: 0.9960821866989136
, x: 0.5301564931869507
y: 0.3696141541004181
z: -0.06299078464508057
visibility: 0.9959152340888977
, x: 0.5289925336837769
y: 0.36883196234703064
z: -0.06299350410699844
visibility: 0.9958778619766235
, x: 0.5276180505752563
y: 0.3679274618625641
z: -0.06303626298904419
visibility: 0.9956432580947876
, x: 0.5235837697982788
y: 0.37157368659973145
z: 0.0005579307326115668
visibility: 0.9921568036079407
, x: 0.5197810530662537
y: 0.36836913228034973
z: -0.07216344773769379
visibility: 0.9867970943450928
, x: 0.5302324295043945
y: 0.3875463604927063
z: -0.03533504158258438
visibility: 0.99

[x: 0.5316007733345032
y: 0.3795291781425476
z: -0.11561347544193268
visibility: 0.9982581734657288
, x: 0.5302473902702332
y: 0.37103986740112305
z: -0.10342065244913101
visibility: 0.9969983696937561
, x: 0.5303969383239746
y: 0.37110668420791626
z: -0.10343052446842194
visibility: 0.9971460700035095
, x: 0.5306217670440674
y: 0.3711743652820587
z: -0.1034332886338234
visibility: 0.997329592704773
, x: 0.5286262035369873
y: 0.36999109387397766
z: -0.11436804383993149
visibility: 0.9973247051239014
, x: 0.5277968645095825
y: 0.36929067969322205
z: -0.1143670529127121
visibility: 0.9972262978553772
, x: 0.5258615612983704
y: 0.36847996711730957
z: -0.11439228057861328
visibility: 0.9969844222068787
, x: 0.5270273089408875
y: 0.37397995591163635
z: -0.031108802184462547
visibility: 0.9946798086166382
, x: 0.5180492401123047
y: 0.36932986974716187
z: -0.08307380974292755
visibility: 0.9917071461677551
, x: 0.5287682414054871
y: 0.388569712638855
z: -0.08717739582061768
visibility: 0.9972

KeyboardInterrupt: 

In [15]:
pl_list

[[0, 0.48307690024375916, 0.4967958331108093],
 [1, 0.48472118377685547, 0.498517781496048],
 [2, 0.48652735352516174, 0.4990350008010864],
 [3, 0.4883361756801605, 0.49949467182159424],
 [4, 0.4807170331478119, 0.49929922819137573],
 [5, 0.479480117559433, 0.5003727674484253],
 [6, 0.4782119691371918, 0.5014791488647461],
 [7, 0.4953361451625824, 0.5024909973144531],
 [8, 0.4921305179595947, 0.5025776624679565],
 [9, 0.48891931772232056, 0.4975716769695282],
 [10, 0.4837947487831116, 0.49901488423347473],
 [11, 0.5095258355140686, 0.5079656839370728],
 [12, 0.48848676681518555, 0.505199670791626],
 [13, 0.5015566349029541, 0.5659513473510742],
 [14, 0.4832293391227722, 0.5690683722496033],
 [15, 0.4971042275428772, 0.6708592176437378],
 [16, 0.49135810136795044, 0.6882834434509277],
 [17, 0.5074352622032166, 0.6446080207824707],
 [18, 0.47750401496887207, 0.6780664324760437],
 [19, 0.5035505890846252, 0.647128701210022],
 [20, 0.49297088384628296, 0.6945905685424805],
 [21, 0.50018233